In [12]:
import numpy as np
import pandas as pd
import glob
import os
from scipy.spatial.transform import Rotation as R

In [70]:
df = pd.read_json('/media/dylan/Elements/nbc/raw/1_1a_task1/spatial.json', orient='index')

In [71]:
def drop_missing(df):
    obj_ids = df['id'].unique()
    drop_steps = []
    for step, group in df.groupby('step'):
        if not len(group) == len(obj_ids):
            print(step)
            drop_steps.append(step)
    df = df[~df['step'].isin(drop_steps)]
    return df
df = drop_missing(df)

In [77]:
def correctRelVel(df):
    head_rots = {}
    for _, row in df[df['name'] == 'Head'].iterrows():
        vel = row[['rotX', 'rotY', 'rotZ', 'rotW']].to_numpy(dtype=float)
        head_rots[row['step']] = R.from_quat(vel)
    def get_rel_vel(row):
        head_rot = head_rots[row['step']]
        vel = row[['velX', 'velY', 'velZ']].to_numpy(dtype=float)
        rel_vel = head_rot.apply(vel, inverse=True)
        row['relVelX'] = rel_vel[0]
        row['relVelY'] = rel_vel[1]
        row['relVelZ'] = rel_vel[2]
        return row
    df = df.apply(lambda row: get_rel_vel(row), axis=1)
    return df
df = correctRelVel(df)

In [78]:
paths = glob.glob('/media/dylan/Elements/nbc/raw/*')
for path in paths:
    print(path)
    path = path + '/spatial.json'
    df = pd.read_json(path, orient='index')
    if not os.path.exists(path.replace('.json', '.json.BAK')):
        df.to_json(path.replace('.json', '.json.BAK'), orient='index')
    df = drop_missing(df)
    df = correctRelVel(df)
    df.to_json(path, orient='index')

/media/dylan/Elements/nbc/raw/10_2b_task1
/media/dylan/Elements/nbc/raw/10_2b_task2
/media/dylan/Elements/nbc/raw/10_2b_task3
/media/dylan/Elements/nbc/raw/10_2b_task4
/media/dylan/Elements/nbc/raw/10_2b_task5
/media/dylan/Elements/nbc/raw/10_2b_task6
/media/dylan/Elements/nbc/raw/11_1c_task1
/media/dylan/Elements/nbc/raw/11_1c_task2
/media/dylan/Elements/nbc/raw/11_1c_task3
/media/dylan/Elements/nbc/raw/11_1c_task4
/media/dylan/Elements/nbc/raw/11_1c_task5
/media/dylan/Elements/nbc/raw/11_1c_task6
/media/dylan/Elements/nbc/raw/2_2a_task1
/media/dylan/Elements/nbc/raw/2_2a_task2
/media/dylan/Elements/nbc/raw/2_2a_task3
/media/dylan/Elements/nbc/raw/2_2a_task4
/media/dylan/Elements/nbc/raw/2_2a_task5
/media/dylan/Elements/nbc/raw/2_2a_task6
/media/dylan/Elements/nbc/raw/3_1b_task1
/media/dylan/Elements/nbc/raw/3_1b_task2
/media/dylan/Elements/nbc/raw/3_1b_task4
/media/dylan/Elements/nbc/raw/3_1b_task5
/media/dylan/Elements/nbc/raw/3_1b_task6
/media/dylan/Elements/nbc/raw/4_2b_task1
/med